### 1.2 过程和它所产生的计算过程

上一节学习了基本算数运算符，将它们进行组合，并通过抽象方法将其定义为组合过程。然而就像是学习了象棋中的基本规则一样，我们对于典型的开局、策略等还没有经验，能够准确预料不同过程所产生的计算过程及结果是成为专家的关键。

#### 1.2.1 线性递归和迭代

考虑阶乘的计算：

$$n! = n*(n-1)*(n-2)...3*2*1$$

其中一种计算方法是：

$$n! = n*(n-1)!$$

In [1]:
(define (factorial n)
  (if (= n 1)
      1
      (* n (factorial (- n 1)))))
(factorial 6)

720

利用1.1.5中提到的替换模型，我们可以观察`(fractorial 6)`的执行过程如下图所示：

![figure 1.3](imgs/figure-1.3.png)

另一种计算方法，从1到n依次相乘，记录每次相乘的乘积：

In [2]:
(define (factorial n)
  (fact-iter 1 1 n))

(define (fact-iter prod counter max-count)
  (if (> counter max-count)
      prod
      (fact-iter (* counter prod)
                 (+ counter 1)
                 max-count)))

(factorial 6)

720

利用替换模型，再次观察`(fractorial 6)`的执行过程如下图所示：

![figure 1.4](imgs/figure-1.4.png)

---

比较两种计算方法，第一种方法称为**递归计算过程（recursive process）**，先通过扩展形成一条推迟计算的链条，完全展开后再逐步缩短完成真正的计算过程。完成这一计算过程需要解释器保存链条前段被推迟的计算。以`n!`的计算为例，解释器所需要保存的信息与`n`成正比，因此这类计算过程又称为**线性递归计算过程（linear recursive process）**。

第二种方法没有用到先扩展后收缩的方式，而是通过记录每一步运算中`n`、`max-count`、`counter`、`prod`的值，这种方法称为**迭代计算过程（iterative process）**，迭代计算过程利用几个**状态变量（state variables）**和一些规则来描述不同状态之间变化、更新的过程，并决定计算过程停止的条件。`n!`所需的迭代步骤随着`n`的增长线性增加，因此这类计算过程也称为**线性迭代计算过程（linear iterative process）**。

---

这里必须明确区分**计算过程 process**与**过程 procedure**的差别，procedure是指语法形式上的，而process是指计算过程的演化；这两者的差异容易让人弄混的地方在于，在一般语言的实现中，递归计算过程所消耗的内存资源随着过程调用的增加而增加，……，After Ch5 Here。

#### 1.2.2 树形递归

另一种常见的计算模式称为**树形递归（tree recursion）**，例如计算斐波那契数列：

$$ Fib(n) =\begin{cases}0 & n = 0,\\1 & n = 1,\\Fib(n-1)+Fib(n-2) & n > 1.\end{cases} $$

In [3]:
(define (fib n)
  (cond ((= n 0) 0)
        ((= n 1) 1)
        (else (+ 
               (fib (- n 1))
               (fib (- n 2))))))
(fib 5)

5

`(fib 5)`的计算过程如下图所示：

![figure-1.5](imgs/figure-1.5.png)

从上图可以看出，在计算过程中存在许多重复计算的步骤，`(fib 3)`的计算几乎占据了其中的一半。

我们也可以用迭代的方式来计算斐波那契数列，先设定$a$和$b$的初始值：

$$a: Fib(1) = 1; b: Fib(0) = 0$$

然后重复下面的计算过程：

$$\begin{cases}a \leftarrow a+b.\\b \leftarrow a.\end{cases}$$

经过$n$次迭代之后，$a$和$b$将分别等于$Fib(n+1)$和$Fib(n)$：

In [4]:
(define (fab n)
  (fib-iter 1 0 n))

; fib-iter
(define (fib-iter a b n)
  (if (= b 0)
      b
      (fib-iter (+ b a) a (- n 1))))
(fib 5)

5

上面这一方法的计算过程是线性迭代的。

**树形递归实例：找零**

有面值为50，25，10，5，1的零钱，一共有多少种方法可以凑成100？用n种面值凑成总额为a的方法，可以分解为下面两种：

1. 除去一种面值（d）剩下所有面值可凑成a的方法数；
2. 至少用到一次第一种面值（d），然后凑成a的方法数$ \Longleftrightarrow $用n种面值凑成（a-d）的方法数。

我们可以根据上面的分解方法，将这一问题递归地转换成较小总额的问题，我们可以根据下面的描述来完成这一算法：

1. 如果总额$a=0$，则只有$1$种方法；
2. 如果$a<0$，则只有$0$种方法；
3. 如果面值种类$n<0$，则只有$0$种方法。

In [7]:
(define (count-change amount)
  (cc amount 5))
(define (cc amount kinds-of-coins)
  (cond ((= amount 0) 1)
        ((or (< amount 0)
             (= kinds-of-coins 0))
         0)
        (else (+ (cc amount (- kinds-of-coins 1))
                 (cc (- amount (first-denomination kinds-of-coins))
                     kinds-of-coins)))))
(define (first-denomination kinds-of-coins)
  (cond ((= kinds-of-coins 1) 1)
        ((= kinds-of-coins 2) 5)
        ((= kinds-of-coins 3) 10)
        ((= kinds-of-coins 4) 25)
        ((= kinds-of-coins 5) 50)))

(count-change 100)

292

#### 1.2.3 增长的阶

上一节中计算斐波那契数列用到了树形递归和线性迭代两种计算过程，两者消耗资源数量是不同的。一种常用的衡量计算过程资源消耗多少的方法称为**增长的阶（order of growth）**。